In [3]:
# importacion general de librerias y de visualizacion (matplotlib y seaborn)
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
import calendar

%matplotlib inline

plt.style.use('default') # haciendo los graficos un poco mas bonitos en matplotlib
#plt.rcParams['figure.figsize'] = (20, 10)

sns.set(style="whitegrid") # seteando tipo de grid en seaborn

pd.options.display.float_format = '{:20,.2f}'.format # suprimimos la notacion cientifica en los outputs

import warnings
warnings.filterwarnings('ignore')

In [4]:
# %timeit sirve para evaluar el tiempo de ejecucion
events = pd.read_csv('events_up_to_01062018.csv')
label_train = pd.read_csv('labels_training_set.csv')
test = pd.read_csv('trocafone_kaggle_test.csv')

#projects_2017.head()trocafone_kaggle_test
#projects_2017

In [40]:
label_train.head()

,person,label
0,0566e9c1,0
1,6ec7ee77,0
2,abe7a2fb,0
3,34728364,0
4,87ed62de,0


In [41]:
test.head()

,person
0,4886f805
1,0297fc1e
2,2d681dd8
3,cccea85e
4,4c8a8b93


In [8]:
events_test = events.merge(test,on='person',how='inner')


In [42]:
events_test.head()

,timestamp,event,person,url,sku,model,condition,storage,color,skus,...,search_engine,channel,new_vs_returning,city,region,country,device_type,screen_resolution,operating_system_version,browser_version
0,2018-05-18 00:11:59,viewed product,4886f805,NaN,"9,288.00",Samsung Galaxy J7 Prime,Excelente,32GB,Dourado,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2018-05-18 00:30:30,viewed product,4886f805,NaN,"9,288.00",Samsung Galaxy J7 Prime,Excelente,32GB,Dourado,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2018-05-18 00:07:23,search engine hit,4886f805,NaN,nan,NaN,NaN,NaN,NaN,NaN,...,Google,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2018-05-18 00:11:56,checkout,4886f805,NaN,"9,288.00",Samsung Galaxy J7 Prime,Excelente,32GB,Dourado,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2018-05-18 00:11:35,viewed product,4886f805,NaN,"9,287.00",Samsung Galaxy J7 Prime,Muito Bom,32GB,Dourado,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
 #feature engine trucho
events_training = events.merge(label_train,on='person',how='inner')
events_training.head()

,timestamp,event,person,url,sku,model,condition,storage,color,skus,...,channel,new_vs_returning,city,region,country,device_type,screen_resolution,operating_system_version,browser_version,label
0,2018-05-18 00:11:27,viewed product,ad93850f,NaN,304.00,iPhone 5s,Muito Bom,32GB,Cinza espacial,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1,2018-05-18 00:23:33,viewed product,ad93850f,NaN,318.00,iPhone 5s,Muito Bom,64GB,Prateado,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2,2018-05-18 00:16:10,viewed product,ad93850f,NaN,"5,907.00",iPhone 5s,Bom - Sem Touch ID,16GB,Cinza espacial,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
3,2018-05-18 00:14:55,viewed product,ad93850f,NaN,"6,023.00",iPhone 5s,Bom - Sem Touch ID,16GB,Dourado,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
4,2018-05-18 00:11:26,ad campaign hit,ad93850f,/comprar/iphone/iphone-5s,nan,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0


In [10]:
events_training['date'] = pd.to_datetime(events_training['timestamp'])

In [26]:
 #calculamos la diferencia de tiempo entre eventos
data1=events_training
data1=events_training.loc[:,['person','date']].sort_values('date',ascending=True)
data1['dateaux']=data1['date'].shift(periods=1,axis=0)
data1['personaux']=data1['person'].shift(periods=1,axis=0)
data1['aux']=data1['person']==data1['personaux']
#data1.groupby('person').apply(lambda x: )
data1=data1[data1['aux']==True]
data1['diftime']=(data1['date']-data1['dateaux']).dt.total_seconds()
len(data1)
#aca tenemos la desviacion estandar
# data1=data1.loc[:,['person','diftime']]

# #aca va la desviacion estandar
# datostd=data1.groupby('person').agg({'diftime':np.std})
# datostd.head()

419051

In [27]:
data1=data1[['person','diftime']]
data1.head()

,person,diftime
418276,7c7e0de9,0.00
418293,7c7e0de9,0.00
366474,8af11dbc,0.00
366432,8af11dbc,0.00
366256,8af11dbc,64.00


In [28]:
datamean=data1.groupby('person').mean()

In [29]:
datameanntrain=label_train.merge(datamean,on='person',how='left')

In [30]:
datameanntrain.fillna(0)

,person,label,diftime
0,0566e9c1,0,0.33
1,6ec7ee77,0,0.00
2,abe7a2fb,0,9.46
3,34728364,0,0.67
4,87ed62de,0,0.33
5,db2c4d27,1,10.79
6,cde431db,0,0.50
7,be65035b,0,1.67
8,a4178891,0,0.00
9,d066f64c,0,1.43


In [34]:
datastd=data1.groupby('person').std()

In [35]:
datastddtrain=label_train.merge(datastd,on='person',how='left')

In [36]:
datastddtrain.fillna(0)

,person,label,diftime
0,0566e9c1,0,0.49
1,6ec7ee77,0,0.00
2,abe7a2fb,0,36.69
3,34728364,0,0.52
4,87ed62de,0,0.82
5,db2c4d27,1,35.47
6,cde431db,0,1.22
7,be65035b,0,1.53
8,a4178891,0,0.00
9,d066f64c,0,1.27


In [37]:
datamedian=data1.groupby('person').median()

In [38]:
datamediatrain=label_train.merge(datamedian,on='person',how='left')

In [39]:
datamediatrain.fillna(0)

,person,label,diftime
0,0566e9c1,0,0.00
1,6ec7ee77,0,0.00
2,abe7a2fb,0,1.00
3,34728364,0,1.00
4,87ed62de,0,0.00
5,db2c4d27,1,1.00
6,cde431db,0,0.00
7,be65035b,0,2.00
8,a4178891,0,0.00
9,d066f64c,0,1.00


In [44]:
events_test['date'] = pd.to_datetime(events_test['timestamp'])

In [45]:
 #calculamos la diferencia de tiempo entre eventos para el train
data1=events_test
data1=events_test.loc[:,['person','date']].sort_values('date',ascending=True)
data1['dateaux']=data1['date'].shift(periods=1,axis=0)
data1['personaux']=data1['person'].shift(periods=1,axis=0)
data1['aux']=data1['person']==data1['personaux']
#data1.groupby('person').apply(lambda x: )
data1=data1[data1['aux']==True]
data1['diftime']=(data1['date']-data1['dateaux']).dt.total_seconds()
len(data1)
#aca tenemos la desviacion estandar
# data1=data1.loc[:,['person','diftime']]

# #aca va la desviacion estandar
# datostd=data1.groupby('person').agg({'diftime':np.std})
# datostd.head()

418324

In [50]:
data1=data1[['person','diftime']]
data1.head()

,person,diftime
1167711,0f4e2a4b,0.00
1167726,0f4e2a4b,0.00
1167710,0f4e2a4b,13.00
1167720,0f4e2a4b,"2,145.00"
1167717,0f4e2a4b,0.00


In [51]:
datamean=data1.groupby('person').mean()

In [52]:
datamediatest=test.merge(datamean,on='person',how='left')

In [54]:
# datamediatest.fillna(0)

In [55]:
datastd=data1.groupby('person').std()

In [56]:
datastdtest=test.merge(datastd,on='person',how='left')

In [58]:
# datastdtest.fillna(0)

In [59]:
datamedian=data1.groupby('person').median()

In [60]:
datamediantest=test.merge(datamedian,on='person',how='left')

In [61]:
datamediantest.fillna(0)

,person,diftime
0,4886f805,0.50
1,0297fc1e,4.00
2,2d681dd8,1.00
3,cccea85e,1.00
4,4c8a8b93,1.00
5,29ebb414,0.00
6,3dc1950f,2.00
7,8ea4c165,1.00
8,d8cfe234,0.00
9,d6bc64df,4.00


In [155]:
#feature engieri del test,misma columna train,pero sin label.
 
 

In [137]:
# es entrenar el modelo en  ramdom forest

In [138]:
#formato del train